In [45]:
import pyspark as spark
import numpy as np
import pandas as pd
import sklearn
import geopandas 
import seaborn as sns
import matplotlib.pyplot as plt

In [46]:
vehicles = pd.read_csv('../data/Traffic_Crashes_-_Vehicles.csv')

/Users/winsometang/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (19,21,40,41,42,44,48,49,50,53,55,58,59,61,71) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [47]:
vehicles.describe()

,CRASH_UNIT_ID,UNIT_NO,NUM_PASSENGERS,VEHICLE_ID,VEHICLE_YEAR,OCCUPANT_CNT,CMV_ID,TRAILER1_LENGTH,TRAILER2_LENGTH,TOTAL_VEHICLE_LENGTH,AXLE_CNT
count,1.057474e+06,1.057474e+06,158436.000000,1.033331e+06,865317.000000,1.033331e+06,11123.000000,1847.000000,49.000000,2222.000000,3219.000000
mean,5.642652e+05,5.134954e+00,1.474842,5.373088e+05,2013.451148,1.083948e+00,6198.145644,47.964266,45.204082,52.823582,11.803976
std,3.276157e+05,3.673931e+03,1.072623,3.094255e+05,152.780094,7.931411e-01,3599.640927,15.593072,30.024975,33.124591,458.362863
min,2.000000e+00,0.000000e+00,1.000000,2.000000e+00,1900.000000,0.000000e+00,1.000000,1.000000,2.000000,1.000000,1.000000
25%,2.790312e+05,1.000000e+00,1.000000,2.696445e+05,2006.000000,1.000000e+00,3079.500000,42.500000,24.000000,35.000000,2.000000
50%,5.645995e+05,2.000000e+00,1.000000,5.378600e+05,2012.000000,1.000000e+00,6125.000000,53.000000,48.000000,53.000000,3.000000
75%,8.490518e+05,2.000000e+00,2.000000,8.059005e+05,2015.000000,1.000000e+00,9325.500000,53.000000,53.000000,65.000000,5.000000
max,1.129364e+06,3.778035e+06,59.000000,1.071292e+06,9999.000000,9.900000e+01,12434.000000,200.000000,123.000000,999.000000,26009.000000


In [48]:
vehicles.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1057474 entries, 0 to 1057473
Data columns (total 72 columns):
 #   Column                    Non-Null Count    Dtype  
---  ------                    --------------    -----  
 0   CRASH_UNIT_ID             1057474 non-null  int64  
 1   CRASH_RECORD_ID           1057474 non-null  object 
 2   RD_NO                     1047752 non-null  object 
 3   CRASH_DATE                1057474 non-null  object 
 4   UNIT_NO                   1057474 non-null  int64  
 5   UNIT_TYPE                 1055922 non-null  object 
 6   NUM_PASSENGERS            158436 non-null   float64
 7   VEHICLE_ID                1033331 non-null  float64
 8   CMRC_VEH_I                19614 non-null    object 
 9   MAKE                      1033326 non-null  object 
 10  MODEL                     1033183 non-null  object 
 11  LIC_PLATE_STATE           943987 non-null   object 
 12  VEHICLE_YEAR              865317 non-null   float64
 13  VEHICLE_DEFECT            1

In [49]:
vehicles.head(10)

,CRASH_UNIT_ID,CRASH_RECORD_ID,RD_NO,CRASH_DATE,UNIT_NO,UNIT_TYPE,NUM_PASSENGERS,VEHICLE_ID,CMRC_VEH_I,MAKE,...,TRAILER1_LENGTH,TRAILER2_LENGTH,TOTAL_VEHICLE_LENGTH,AXLE_CNT,VEHICLE_CONFIG,CARGO_BODY_TYPE,LOAD_TYPE,HAZMAT_OUT_OF_SERVICE_I,MCS_OUT_OF_SERVICE_I,HAZMAT_CLASS
0,829999,24ddf9fd8542199d832e1c223cc474e5601b356f1d77a6...,JD124535,01/22/2020 06:25:00 AM,1,DRIVER,NaN,796949.0,NaN,INFINITI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,749947,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,JC451435,09/28/2019 03:30:00 AM,1,DRIVER,NaN,834816.0,NaN,HONDA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,749949,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,JC451435,09/28/2019 03:30:00 AM,2,PARKED,NaN,834819.0,NaN,TOYOTA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,749950,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,JC451435,09/28/2019 03:30:00 AM,3,PARKED,NaN,834817.0,NaN,GENERAL MOTORS CORPORATION (GMC),...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,871921,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,JD208731,04/13/2020 10:50:00 PM,2,DRIVER,NaN,827212.0,NaN,BUICK,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,473653,80f5b6eff349a72094c9c7a11745549caa8bd39b57cedd...,JB428391,09/09/2018 08:50:00 AM,2,PARKED,NaN,453462.0,NaN,LINCOLN-CONTINENTAL,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,834812,ecf9e646f89a73ed3df0f373e4fbd18e91f5a595004019...,JD138756,02/03/2020 03:45:00 PM,1,DRIVER,NaN,792247.0,NaN,HONDA,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,834813,ecf9e646f89a73ed3df0f373e4fbd18e91f5a595004019...,JD138756,02/03/2020 03:45:00 PM,2,DRIVER,1.0,792252.0,NaN,SUBARU,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,837380,c7c57a49274b3aa953d0b76346f68e116ed8c3d72a683a...,JD143282,02/06/2020 12:00:00 AM,1,DRIVER,NaN,794648.0,NaN,UNKNOWN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,837381,c7c57a49274b3aa953d0b76346f68e116ed8c3d72a683a...,JD143282,02/06/2020 12:00:00 AM,2,PARKED,NaN,794653.0,NaN,INFINITI,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [50]:
vehicles.isnull().sum().head(19)

CRASH_UNIT_ID             0
CRASH_RECORD_ID           0
RD_NO                  9722
CRASH_DATE                0
UNIT_NO                   0
UNIT_TYPE              1552
NUM_PASSENGERS       899038
VEHICLE_ID            24143
CMRC_VEH_I          1037860
MAKE                  24148
MODEL                 24291
LIC_PLATE_STATE      113487
VEHICLE_YEAR         192157
VEHICLE_DEFECT        24143
VEHICLE_TYPE          24143
VEHICLE_USE           24143
TRAVEL_DIRECTION      24143
MANEUVER              24143
TOWED_I              935389
dtype: int64

In [51]:
vehicles['VEHICLE_DEFECT'].value_counts()

NONE                596378
UNKNOWN             426401
OTHER                 5114
BRAKES                3509
TIRES                  513
STEERING               477
WHEELS                 267
SUSPENSION             175
ENGINE/MOTOR           134
FUEL SYSTEM            116
LIGHTS                  76
WINDOWS                 66
CARGO                   33
SIGNALS                 27
RESTRAINT SYSTEM        16
TRAILER COUPLING        16
EXHAUST                 13
Name: VEHICLE_DEFECT, dtype: int64

In [52]:
vehicles['FIRE_I'].value_counts()

Y    436
N    346
Name: FIRE_I, dtype: int64

In [53]:
vehicles['MANEUVER'].value_counts()

STRAIGHT AHEAD                        467595
PARKED                                140852
SLOW/STOP IN TRAFFIC                   81014
UNKNOWN/NA                             80557
TURNING LEFT                           59746
BACKING                                44057
TURNING RIGHT                          33971
PASSING/OVERTAKING                     24424
CHANGING LANES                         20569
OTHER                                  17225
ENTERING TRAFFIC LANE FROM PARKING     12201
MERGING                                 7126
STARTING IN TRAFFIC                     6183
U-TURN                                  5602
LEAVING TRAFFIC LANE TO PARK            5250
AVOIDING VEHICLES/OBJECTS               4552
SKIDDING/CONTROL LOSS                   4438
ENTER FROM DRIVE/ALLEY                  4343
SLOW/STOP - LEFT TURN                   3042
PARKED IN TRAFFIC LANE                  2999
SLOW/STOP - RIGHT TURN                  1928
SLOW/STOP - LOAD/UNLOAD                 1663
NEGOTIATIN

In [54]:
clean_vehicles = vehicles[['CRASH_UNIT_ID',"CRASH_RECORD_ID","CRASH_DATE",'MAKE','MODEL','VEHICLE_TYPE','VEHICLE_DEFECT','TRAVEL_DIRECTION','MANEUVER','FIRST_CONTACT_POINT','FIRE_I','VEHICLE_DEFECT']]

In [55]:
clean_vehicles

,CRASH_UNIT_ID,CRASH_RECORD_ID,CRASH_DATE,MAKE,MODEL,VEHICLE_TYPE,VEHICLE_DEFECT,TRAVEL_DIRECTION,MANEUVER,FIRST_CONTACT_POINT,FIRE_I,VEHICLE_DEFECT
0,829999,24ddf9fd8542199d832e1c223cc474e5601b356f1d77a6...,01/22/2020 06:25:00 AM,INFINITI,UNKNOWN,PASSENGER,NONE,N,STRAIGHT AHEAD,FRONT,NaN,NONE
1,749947,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,09/28/2019 03:30:00 AM,HONDA,CIVIC,PASSENGER,UNKNOWN,N,STRAIGHT AHEAD,FRONT,NaN,UNKNOWN
2,749949,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,09/28/2019 03:30:00 AM,TOYOTA,YARIS,UNKNOWN/NA,NONE,N,PARKED,ROOF,NaN,NONE
3,749950,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,09/28/2019 03:30:00 AM,GENERAL MOTORS CORPORATION (GMC),SIERRA,UNKNOWN/NA,UNKNOWN,N,PARKED,ROOF,NaN,UNKNOWN
4,871921,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,04/13/2020 10:50:00 PM,BUICK,ENCORE,PASSENGER,NONE,W,STRAIGHT AHEAD,FRONT-RIGHT,NaN,NONE
...,...,...,...,...,...,...,...,...,...,...,...,...
1057469,1119458,18cac74e5599280b7925e576d6bb13f0baf4742f258cd7...,06/07/2021 10:00:00 AM,UNKNOWN,OTHER (EXPLAIN IN NARRATIVE),PASSENGER,UNKNOWN,UNKNOWN,UNKNOWN/NA,UNKNOWN,NaN,UNKNOWN
1057470,1119459,18cac74e5599280b7925e576d6bb13f0baf4742f258cd7...,06/07/2021 10:00:00 AM,HYUNDAI,ELANTRA,PASSENGER,NONE,UNKNOWN,PARKED,SIDE-LEFT,NaN,NONE
1057471,1120654,95fabe6828580c5032c5b78ffae40df686df2ecc97c90e...,06/07/2021 04:30:00 PM,CHEVROLET,TAHOE,PASSENGER,NONE,UNKNOWN,UNKNOWN/NA,UNKNOWN,NaN,NONE
1057472,1120655,95fabe6828580c5032c5b78ffae40df686df2ecc97c90e...,06/07/2021 04:30:00 PM,MITSUBISHI,GALANT,PASSENGER,NONE,N,PARKED,OTHER,NaN,NONE


In [56]:
clean_vehicles["TRAVEL_DIRECTION"].value_counts()

N          243678
S          237525
W          216393
E          211328
UNKNOWN     80485
SE          12838
NW          11703
SW           9709
NE           9672
Name: TRAVEL_DIRECTION, dtype: int64

In [57]:
clean_vehicles['TRAVEL_DIRECTION'] = clean_vehicles['TRAVEL_DIRECTION'].fillna('UNKNOWN')

/var/folders/nt/rg8_jkq90v77r8l8dc_9wl0m0000gn/T/ipykernel_17752/3910227358.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_vehicles['TRAVEL_DIRECTION'] = clean_vehicles['TRAVEL_DIRECTION'].fillna('UNKNOWN')


In [58]:
clean_vehicles['MANEUVER'] = clean_vehicles['MANEUVER'].fillna('UNKNOWN/NA')

/var/folders/nt/rg8_jkq90v77r8l8dc_9wl0m0000gn/T/ipykernel_17752/3741832184.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_vehicles['MANEUVER'] = clean_vehicles['MANEUVER'].fillna('UNKNOWN/NA')


In [60]:
clean_vehicles.isnull().sum()


CRASH_UNIT_ID                0
CRASH_RECORD_ID              0
CRASH_DATE                   0
MAKE                     24148
MODEL                    24291
VEHICLE_TYPE             24143
VEHICLE_DEFECT           24143
TRAVEL_DIRECTION             0
MANEUVER                     0
FIRST_CONTACT_POINT      31642
FIRE_I                 1056692
VEHICLE_DEFECT           24143
dtype: int64

In [61]:
clean_vehicles[clean_vehicles.isna().any(axis=1)]

,CRASH_UNIT_ID,CRASH_RECORD_ID,CRASH_DATE,MAKE,MODEL,VEHICLE_TYPE,VEHICLE_DEFECT,TRAVEL_DIRECTION,MANEUVER,FIRST_CONTACT_POINT,FIRE_I,VEHICLE_DEFECT
0,829999,24ddf9fd8542199d832e1c223cc474e5601b356f1d77a6...,01/22/2020 06:25:00 AM,INFINITI,UNKNOWN,PASSENGER,NONE,N,STRAIGHT AHEAD,FRONT,NaN,NONE
1,749947,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,09/28/2019 03:30:00 AM,HONDA,CIVIC,PASSENGER,UNKNOWN,N,STRAIGHT AHEAD,FRONT,NaN,UNKNOWN
2,749949,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,09/28/2019 03:30:00 AM,TOYOTA,YARIS,UNKNOWN/NA,NONE,N,PARKED,ROOF,NaN,NONE
3,749950,81dc0de2ed92aa62baccab641fa377be7feb1cc47e6554...,09/28/2019 03:30:00 AM,GENERAL MOTORS CORPORATION (GMC),SIERRA,UNKNOWN/NA,UNKNOWN,N,PARKED,ROOF,NaN,UNKNOWN
4,871921,af84fb5c8d996fcd3aefd36593c3a02e6e7509eeb27568...,04/13/2020 10:50:00 PM,BUICK,ENCORE,PASSENGER,NONE,W,STRAIGHT AHEAD,FRONT-RIGHT,NaN,NONE
...,...,...,...,...,...,...,...,...,...,...,...,...
1057469,1119458,18cac74e5599280b7925e576d6bb13f0baf4742f258cd7...,06/07/2021 10:00:00 AM,UNKNOWN,OTHER (EXPLAIN IN NARRATIVE),PASSENGER,UNKNOWN,UNKNOWN,UNKNOWN/NA,UNKNOWN,NaN,UNKNOWN
1057470,1119459,18cac74e5599280b7925e576d6bb13f0baf4742f258cd7...,06/07/2021 10:00:00 AM,HYUNDAI,ELANTRA,PASSENGER,NONE,UNKNOWN,PARKED,SIDE-LEFT,NaN,NONE
1057471,1120654,95fabe6828580c5032c5b78ffae40df686df2ecc97c90e...,06/07/2021 04:30:00 PM,CHEVROLET,TAHOE,PASSENGER,NONE,UNKNOWN,UNKNOWN/NA,UNKNOWN,NaN,NONE
1057472,1120655,95fabe6828580c5032c5b78ffae40df686df2ecc97c90e...,06/07/2021 04:30:00 PM,MITSUBISHI,GALANT,PASSENGER,NONE,N,PARKED,OTHER,NaN,NONE


In [62]:
clean_vehicles["FIRE_I"]= clean_vehicles["FIRE_I"].fillna('N')

/var/folders/nt/rg8_jkq90v77r8l8dc_9wl0m0000gn/T/ipykernel_17752/2741210271.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_vehicles["FIRE_I"]= clean_vehicles["FIRE_I"].fillna('N')


In [63]:
clean_vehicles["FIRE_I"].value_counts()

N    1057038
Y        436
Name: FIRE_I, dtype: int64

In [65]:
clean_vehicles[clean_vehicles['MODEL'].isna()]


,CRASH_UNIT_ID,CRASH_RECORD_ID,CRASH_DATE,MAKE,MODEL,VEHICLE_TYPE,VEHICLE_DEFECT,TRAVEL_DIRECTION,MANEUVER,FIRST_CONTACT_POINT,FIRE_I,VEHICLE_DEFECT
11,837501,55a20437d79a3176ac805c65b13940186970246ab14ce5...,02/07/2020 01:19:00 PM,NaN,NaN,NaN,NaN,UNKNOWN,UNKNOWN/NA,NaN,N,NaN
26,946687,af61b8eabb1b375bd1f6ff97f6ecce3e0f3592d4930381...,08/27/2020 04:20:00 PM,NaN,NaN,NaN,NaN,UNKNOWN,UNKNOWN/NA,NaN,N,NaN
40,473735,034f42deaec11021e28cda25717373e36351025652ccf1...,09/09/2018 12:25:00 PM,NaN,NaN,NaN,NaN,UNKNOWN,UNKNOWN/NA,NaN,N,NaN
45,950910,00aa496fc5a12fc3feeb610857cab1c2751d384ffbefbb...,09/03/2020 04:15:00 AM,NaN,NaN,NaN,NaN,UNKNOWN,UNKNOWN/NA,NaN,N,NaN
58,913811,f84c48d4194761d1bb3808061f35ebf796508fb209deb5...,07/07/2020 02:50:00 PM,NaN,NaN,NaN,NaN,UNKNOWN,UNKNOWN/NA,NaN,N,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
1057233,1112297,a4259ad3df81c131a3e856abad4ae1feebfdc4fd27b0d2...,06/07/2021 02:40:00 PM,NaN,NaN,NaN,NaN,UNKNOWN,UNKNOWN/NA,NaN,N,NaN
1057290,1112426,9f05cfa8b1c663b63922ea6587a4dfe7f2dd06ab66300e...,06/07/2021 05:35:00 PM,NaN,NaN,NaN,NaN,UNKNOWN,UNKNOWN/NA,NaN,N,NaN
1057370,1112550,8410d8a4d9c8ad18bfca5eb386eb1a9da75b33bee16b5c...,06/07/2021 09:30:00 PM,NaN,NaN,NaN,NaN,UNKNOWN,UNKNOWN/NA,NaN,N,NaN
1057404,1112597,f1ce72f60a0243a7de841847651b4366fc262e4e7b18d5...,06/07/2021 11:20:00 PM,NaN,NaN,NaN,NaN,UNKNOWN,UNKNOWN/NA,NaN,N,NaN


In [66]:
clean_vehicles["MAKE"]= clean_vehicles["MAKE"].fillna('NONE')

/var/folders/nt/rg8_jkq90v77r8l8dc_9wl0m0000gn/T/ipykernel_17752/169736604.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_vehicles["MAKE"]= clean_vehicles["MAKE"].fillna('NONE')


In [67]:
clean_vehicles["MODEL"]= clean_vehicles["MODEL"].fillna('NONE')

/var/folders/nt/rg8_jkq90v77r8l8dc_9wl0m0000gn/T/ipykernel_17752/2443657553.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_vehicles["MODEL"]= clean_vehicles["MODEL"].fillna('NONE')


In [68]:
clean_vehicles["VEHICLE_TYPE"]= clean_vehicles["VEHICLE_TYPE"].fillna('NONE')

/var/folders/nt/rg8_jkq90v77r8l8dc_9wl0m0000gn/T/ipykernel_17752/1188925135.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_vehicles["VEHICLE_TYPE"]= clean_vehicles["VEHICLE_TYPE"].fillna('NONE')


In [69]:
clean_vehicles['VEHICLE_DEFECT'] = clean_vehicles["VEHICLE_DEFECT"].fillna("NONE")

/var/folders/nt/rg8_jkq90v77r8l8dc_9wl0m0000gn/T/ipykernel_17752/3804530600.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_vehicles['VEHICLE_DEFECT'] = clean_vehicles["VEHICLE_DEFECT"].fillna("NONE")


In [70]:
clean_vehicles['FIRST_CONTACT_POINT'] = clean_vehicles["FIRST_CONTACT_POINT"].fillna("NONE")

/var/folders/nt/rg8_jkq90v77r8l8dc_9wl0m0000gn/T/ipykernel_17752/544024656.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_vehicles['FIRST_CONTACT_POINT'] = clean_vehicles["FIRST_CONTACT_POINT"].fillna("NONE")


In [71]:
clean_vehicles.isnull().sum()

CRASH_UNIT_ID          0
CRASH_RECORD_ID        0
CRASH_DATE             0
MAKE                   0
MODEL                  0
VEHICLE_TYPE           0
VEHICLE_DEFECT         0
TRAVEL_DIRECTION       0
MANEUVER               0
FIRST_CONTACT_POINT    0
FIRE_I                 0
VEHICLE_DEFECT         0
dtype: int64

In [ ]:
clean_vehicles.columns

In [ ]:
dum_clean_vehicles = pd.get_dummies(clean_vehicles,columns = ['MAKE', 'MODEL',
       'VEHICLE_TYPE', 'VEHICLE_DEFECT', 'FIRST_CONTACT_POINT', 'FIRE_I'], drop_first = True)

In [ ]:
clean_vehicles.to_csv("clean_vehicles.csv")